In [1]:
from main import *
from ai import *
import pandas as pd
import numpy as np

In [2]:
# import sys
# sys.setrecursionlimit(15000)

In [3]:
df = pd.read_csv('data/dataset.csv').drop('Unnamed: 0', axis=1)
df.head()

,Id,Sudoku,Solution,Level,People,Average-Time
0,0,8...74...1.........7.5.9.4....65.4..73.....296...,8591742361648325973725698412916574837354186296...,Diabolical,254,25
1,1,.5.....7.9......6...21.75.8.4..69.3...8...4......,8539461729175823644621375981458692376283754197...,Moderate,281,12
2,2,...45.2..3.........8....4..7...94.3.9..5.2.76....,6974532183248617955819274637186945329435128762...,Diabolical,265,22
3,3,9......31.3...8.....5.7.9..3...4...8.916..54.4...,9274568316341982758153729643527496187916835424...,Tough,279,19
4,4,9.......2784.1.5.....4.....2.53.1.7...7...8......,9615387427846125395234796182953814761479568236...,Moderate,301,14


In [4]:
#solve(df['Sudoku'][0])
solve_technique(df['Sudoku'][0],'single_position')[0]

'Not solved'

In [5]:
solve_technique(df['Sudoku'][0],'single_candidate')[0]

'Not solved'

In [6]:
solve_technique(df['Sudoku'][0],'naked_twins')[0]

'Not solved'

In [7]:
solve(df['Sudoku'][0])

('Solved',
 {'A1': '8',
  'A2': '5',
  'A3': '9',
  'A4': '1',
  'A5': '7',
  'A6': '4',
  'A7': '2',
  'A8': '3',
  'A9': '6',
  'B1': '1',
  'B2': '6',
  'B3': '4',
  'B4': '8',
  'B5': '3',
  'B6': '2',
  'B7': '5',
  'B8': '9',
  'B9': '7',
  'C1': '3',
  'C2': '7',
  'C3': '2',
  'C4': '5',
  'C5': '6',
  'C6': '9',
  'C7': '8',
  'C8': '4',
  'C9': '1',
  'D1': '2',
  'D2': '9',
  'D3': '1',
  'D4': '6',
  'D5': '5',
  'D6': '7',
  'D7': '4',
  'D8': '8',
  'D9': '3',
  'E1': '7',
  'E2': '3',
  'E3': '5',
  'E4': '4',
  'E5': '1',
  'E6': '8',
  'E7': '6',
  'E8': '2',
  'E9': '9',
  'F1': '6',
  'F2': '4',
  'F3': '8',
  'F4': '2',
  'F5': '9',
  'F6': '3',
  'F7': '1',
  'F8': '7',
  'F9': '5',
  'G1': '9',
  'G2': '2',
  'G3': '6',
  'G4': '3',
  'G5': '8',
  'G6': '1',
  'G7': '7',
  'G8': '5',
  'G9': '4',
  'H1': '4',
  'H2': '1',
  'H3': '3',
  'H4': '7',
  'H5': '2',
  'H6': '5',
  'H7': '9',
  'H8': '6',
  'H9': '8',
  'I1': '5',
  'I2': '8',
  'I3': '7',
  'I4': '9',
 

In [8]:
import json
a =solve(df['Sudoku'][0])[1]

In [9]:
"".join([value if len(value) == 1 else "." for value in a.values()])

'859174236164832597372569841291657483735418629648293175926381754413725968587946312'

In [10]:
rows = 'ABCDEFGHI'
cols = '123456789'
boxes = [s+t for s in rows for t in cols]

def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    print

In [11]:
display(a)

8 5 9 |1 7 4 |2 3 6 
1 6 4 |8 3 2 |5 9 7 
3 7 2 |5 6 9 |8 4 1 
------+------+------
2 9 1 |6 5 7 |4 8 3 
7 3 5 |4 1 8 |6 2 9 
6 4 8 |2 9 3 |1 7 5 
------+------+------
9 2 6 |3 8 1 |7 5 4 
4 1 3 |7 2 5 |9 6 8 
5 8 7 |9 4 6 |3 1 2 


In [12]:
df['Solved_single_position'] = df['Sudoku'].apply(lambda x: solve_technique(x, 'single_position')[0])

In [13]:
df['Solved_single_position'].value_counts()

Not solved    4813
Solved         216
Name: Solved_single_position, dtype: int64

In [14]:
df['Solved_single_candidate'] = df['Sudoku'].apply(lambda x: solve_technique(x, 'single_candidate')[0])

In [15]:
df['Solved_single_candidate'].value_counts()

Not solved    5029
Name: Solved_single_candidate, dtype: int64

In [16]:
df['Solved_naked_twins']= df['Sudoku'].apply(lambda x: solve_technique(x, 'naked_twins')[0])

In [17]:
df['Solved_naked_twins'].value_counts()

Not solved    5029
Name: Solved_naked_twins, dtype: int64

In [18]:
df['Solved']= df['Sudoku'].apply(lambda x: solve(x)[0])

In [19]:
df['Solved'].value_counts()

Solved    5029
Name: Solved, dtype: int64

In [20]:
df.head()

,Id,Sudoku,Solution,Level,People,Average-Time,Solved_single_position,Solved_single_candidate,Solved_naked_twins,Solved
0,0,8...74...1.........7.5.9.4....65.4..73.....296...,8591742361648325973725698412916574837354186296...,Diabolical,254,25,Not solved,Not solved,Not solved,Solved
1,1,.5.....7.9......6...21.75.8.4..69.3...8...4......,8539461729175823644621375981458692376283754197...,Moderate,281,12,Not solved,Not solved,Not solved,Solved
2,2,...45.2..3.........8....4..7...94.3.9..5.2.76....,6974532183248617955819274637186945329435128762...,Diabolical,265,22,Not solved,Not solved,Not solved,Solved
3,3,9......31.3...8.....5.7.9..3...4...8.916..54.4...,9274568316341982758153729643527496187916835424...,Tough,279,19,Not solved,Not solved,Not solved,Solved
4,4,9.......2784.1.5.....4.....2.53.1.7...7...8......,9615387427846125395234796182953814761479568236...,Moderate,301,14,Not solved,Not solved,Not solved,Solved


In [21]:
display(solve(df['Sudoku'][0])[1])

8 5 9 |1 7 4 |2 3 6 
1 6 4 |8 3 2 |5 9 7 
3 7 2 |5 6 9 |8 4 1 
------+------+------
2 9 1 |6 5 7 |4 8 3 
7 3 5 |4 1 8 |6 2 9 
6 4 8 |2 9 3 |1 7 5 
------+------+------
9 2 6 |3 8 1 |7 5 4 
4 1 3 |7 2 5 |9 6 8 
5 8 7 |9 4 6 |3 1 2 


In [22]:
display(solve(df['Sudoku'][1])[1])

8 5 3 |9 4 6 |1 7 2 
9 1 7 |5 8 2 |3 6 4 
4 6 2 |1 3 7 |5 9 8 
------+------+------
1 4 5 |8 6 9 |2 3 7 
6 2 8 |3 7 5 |4 1 9 
7 3 9 |4 2 1 |8 5 6 
------+------+------
3 7 1 |2 9 8 |6 4 5 
2 9 4 |6 5 3 |7 8 1 
5 8 6 |7 1 4 |9 2 3 


In [23]:
df['Sudoku'][1]

'.5.....7.9......6...21.75.8.4..69.3...8...4...3.42..5.3.12.86...9......1.8.....2.'

In [26]:
test_wrong = ".55....7.9......6...21.75.8.4..69.3...8...4...3.42..5.3.12.86...9......1.8.....2."

In [27]:
solve(test_wrong)[1]

AttributeError: 'bool' object has no attribute 'keys'

In [29]:
rows = 'ABCDEFGHI'
cols = '123456789'

boxes = [s+t for s in rows for t in cols]

values = dict(zip(boxes, ["." if element == "." else element for element in test_wrong]))
for a in values.values():
    print(a)

.
5
5
.
.
.
.
7
.
9
.
.
.
.
.
.
6
.
.
.
2
1
.
7
5
.
8
.
4
.
.
6
9
.
3
.
.
.
8
.
.
.
4
.
.
.
3
.
4
2
.
.
5
.
3
.
1
2
.
8
6
.
.
.
9
.
.
.
.
.
.
1
.
8
.
.
.
.
.
2
.


In [99]:
# def checker(grid):
    
#     rows = 'ABCDEFGHI'
#     cols = '123456789'

#     boxes = [s+t for s in rows for t in cols]
#     values_grid = dict(zip(boxes, ["." if element == "." else element for element in grid]))
#     row_units = [[s+t for s in r for t in cols] for r in rows]
#     column_units = [[s+t for s in rows for t in c] for c in cols]
#     square_units = [[s+t for s in rs for t in cs]for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
#     unitlist = row_units + column_units + square_units
#     units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
#     peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)
    
# #     print(values_grid.keys())
# #     print('hiiiiiiiii',row_units)
#     if 'A1' in row_units:
#         print(row_units)

# #     for a in values_grid.values():
# #         if a == '.':
# #             print('no')
# #         else:
# #             if a in values_grid[a]:
# #                 print('yes')


In [15]:
def split(element): 
    return [char for char in element]  

def transform_matrix(element):
  return np.reshape(split(element), (-1, 9))

In [15]:
transform_matrix(a)

array([['.', '4', '8', '.', '.', '.', '.', '7', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '7', '3', '2', '.', '6', '5', '1', '.'],
       ['7', '.', '.', '4', '.', '5', '.', '.', '.'],
       ['3', '.', '.', '.', '7', '.', '.', '.', '8'],
       ['.', '.', '.', '8', '.', '3', '.', '.', '5'],
       ['.', '5', '4', '1', '.', '7', '3', '6', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '1', '.', '.', '.', '.', '2', '5', '.']], dtype='<U1')